In [1]:
import numpy as np
import pandas as pd
import os
import glob
import nltk
from collections import Counter
from nltk.probability import FreqDist
import matplotlib.pyplot as plt

%matplotlib inline

## Настройки

In [2]:
#path = './data/Reuters_russian_articles/'
path = './data/2017.12.09-1959-Polarity-Job1943-ru-RU/Reuters_russian_articles/'
window_size = 200

## Пример разметки

In [3]:
filename = os.path.join(path, '10220334-УМВБ - карбованец не изменился в Киеве 176.100 за доллар..csv')
data = pd.read_csv(filename, sep='\t', names=['Offset', 'Text', 'Tags'], skip_blank_lines=False)

In [4]:
data.head(15)

,Offset,Text,Tags
0,Offset=0,Text=МОСКВА,NaN
1,Offset=8,Text=20,NaN
2,Offset=11,Text=авг,NaN
3,Offset=16,Text=Рейтер,NaN
4,Offset=26,Text=Курс,NaN
5,Offset=31,Text=карбованца,NaN
6,Offset=42,Text=к,NaN
7,Offset=44,Text=доллару,NaN
8,Offset=52,Text=не,NaN
9,Offset=55,Text=изменился,Polarity=Negative(4529)


## Размеры данных

### Количество текстов

In [5]:
# Берем все csv-файлы из нужной директории
directory = glob.glob(os.path.join(path, '*.csv'))
len(directory)

17487

In [8]:
# for filename in directory:
    
#     data = pd.read_csv(filename, sep='\t', names=['Offset', 'Text', 'Tags 1', 'Tags 2'], skip_blank_lines=False)
    
#     # Найдем, есть ли еще столбцы, помимо третьего
#     if (not pd.isnull(np.array(data['Tags 2'])).all()):
#         print(data)
        
# # Их нет, поэтому в дальнейшем берем только первые 3

In [9]:
%%time

words = []  # список слов во всех текстах
symbols_count = 0  # количество символов во всех текстах
sentenses_count = 0  # количество предложений во всех текстах
is_new_sentense = True  # индикатор конца предложения, для подсчета количества предложений с тегами
sentenses_with_tags_count = 0  # Количество предложений с тегами
files_count = 0  # счетчик файлов, нужен для вывода прогресса
tags_count = Counter()  # счетчик количества тегов

for filename in directory:
    
    data = pd.read_csv(filename, sep='\t', names=['Offset', 'Text', 'Tags'], skip_blank_lines=False)
        
    for row_number in range(len(data)):
        
        # Предложения разделены пустой строкой, поэтому чтобы посчитать количество 
        # предложений, посчитаем количество пустых строк и потом прибавим 1
        if (pd.isnull(data['Offset'][row_number])):
            sentenses_count += 1
            is_new_sentense = True
            
        # Считаем количество слов и символов
        if (not pd.isnull(data['Text'][row_number])):
            word = data['Text'][row_number].split("=")[1]
            words.append(word.lower())
            symbols_count += len(word)
            
        # Считаем количество тегов
        if (not pd.isnull(data['Tags'][row_number])):
            tags_count[data['Tags'][row_number]] += 1
            if (is_new_sentense):
                sentenses_with_tags_count += 1
                is_new_sentense = False
            
            
    sentenses_count += 1
    files_count += 1
    if files_count % 1000 == 0:
        print(files_count)

1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
CPU times: user 3min 11s, sys: 1.16 s, total: 3min 13s
Wall time: 3min 13s


### Количество слов

In [10]:
len(words)

3222657

### Количество символов

In [11]:
symbols_count

22703708

### Количество предложений

In [12]:
sentenses_count

206856

### Количество тегов

In [13]:
tags_count

Counter({'Polarity=Negative(4529)': 18602,
         'PolarityNonPredicative=NonPredicativeNegative(4511)': 2664,
         'PolarityNonPredicative=NonPredicativeNegative_Initial(6091)': 6})

In [14]:
tags_count['Polarity=Negative(4529)'] / len(words)

0.00577225562633566

In [15]:
tags_count['PolarityNonPredicative=NonPredicativeNegative(4511)'] / len(words)

0.0008266470803439522

### Количество предложений с тегами

In [16]:
sentenses_with_tags_count

18685

In [17]:
sentenses_with_tags_count / sentenses_count

0.09032853772672778

## Исследование разметки

In [18]:
def cat(filename):
    with open(filename, 'r') as f:
        return f.read().replace('\n', '')

In [19]:
%%time

words_in_windows = []  # слова из всех +/-2 окон тегированных
words_without_no = []  # тегированные слова, в контексте которых нет "не" и "нет"
hasnt_negative_word_contexts = []  # контексты этих слов
files_count = 0  # счетчик файлов, нужен для вывода прогресса

for filename in directory:
    
    data = pd.read_csv(filename, sep='\t', names=['Offset', 'Text', 'Tags'], skip_blank_lines=False)
        
    for row_number in range(len(data)):
            
        # Находим тегированное слово
        if (not pd.isnull(data['Tags'][row_number])):
            
            has_no = False
            
            # Смотрим на его +/-2 окно
            for window_step in [-2, -1, 0, 1, 2]:
                if row_number + window_step >= 0 and row_number + window_step < len(data):
                    text = data['Text'][row_number + window_step]
                    if not isinstance(text, float):
                        text = text.split("=")[1].lower()
                        words_in_windows.append(text)
                    
                        if not has_no and (text == "не" or text == "нет"):
                            has_no = True
            
            # Расширяем окно до +/-3 и находим слова с отрицанием, вокруг которых нет "не" и "нет"
            for window_step in [-3, 3]:
                if row_number + window_step >= 0 and row_number + window_step < len(data):
                    text = data['Text'][row_number + window_step]
                    if not has_no and not isinstance(text, float):
                        text = text.split("=")[1].lower()
                        if text == "не" or text == "нет":
                            has_no = True
            
            text = data['Text'][row_number]
            if (not has_no and not isinstance(text, float)):
                text = text.split("=")[1]
                words_without_no.append(data['Text'][row_number].split("=")[1].lower())
                offset = int(data['Offset'][row_number].split("=")[1])
                file_text = cat(filename.replace("csv", "txt"))
                hasnt_negative_word_contexts.append(file_text[offset - int(window_size / 2):offset] + 
                                                   '\x1b[31m' + file_text[offset:offset + len(text)] + 
                                                   '\x1b[0m' + file_text[offset + len(text):offset + 
                                                                         int(window_size / 2)])
            
    files_count += 1
    if files_count % 1000 == 0:
        print(files_count)

1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
CPU times: user 1min 11s, sys: 840 ms, total: 1min 12s
Wall time: 1min 14s


In [20]:
def get_freq_dict(wordlist):
    
    dictionary = FreqDist(wordlist)
    
    import operator
    dictionary = sorted(dictionary.items(), key=operator.itemgetter(1), reverse=True)

    for i in range(len(dictionary)):
        dictionary[i] = (dictionary[i][0], dictionary[i][1] / len(wordlist))
        
    return dictionary

In [21]:
words_in_windows_freq = get_freq_dict(words_in_windows)

### top-5 в окне +/-2 тегированных

In [22]:
words_in_windows_freq[:5]

[('не', 0.19741499879430913),
 ('за', 0.016407041234627443),
 ('рейтер', 0.015789727513865443),
 ('ответственности', 0.014564745599228358),
 ('несет', 0.013243308415722209)]

In [23]:
words_in_windows_freq[:10]

[('не', 0.19741499879430913),
 ('за', 0.016407041234627443),
 ('рейтер', 0.015789727513865443),
 ('ответственности', 0.014564745599228358),
 ('несет', 0.013243308415722209),
 ('и', 0.01288642392090668),
 ('в', 0.012876778394019774),
 ('на', 0.010494333252953942),
 ('пока', 0.009780564263322885),
 ('нет', 0.007726067036411864)]

### Тегированные слова, в контексте +/-3 которых нет "не" и "нет"

In [34]:
len(words_without_no)

55

In [25]:
get_freq_dict(words_without_no)[:5]

[('отменен', 0.05454545454545454),
 ('дробиться', 0.03636363636363636),
 ('проводить', 0.03636363636363636),
 ('премьера', 0.03636363636363636),
 ('иметь', 0.03636363636363636)]

In [26]:
for context in hasnt_negative_word_contexts[:10]:
    print(context.replace("\x1b[31m", "\x1b[35m"))
    print("\n")

атизацию был подготовлен на очень низком уровне. Остается такое впечатление, что он не был до конца продуман",-сказал журналистам редактор независимого азербайджанского журнала "Экономикс" Губад Ибад-


 в среду председатель Мажилиса. "Бюджет на 1997 год мало реален в своей основе, однако такой бюджет лучше чем никакого", - сказал Оспанов (председатель Мажилиса - нижней палаты парламента). Правительс


) были приняты незаконно". В июне этого года Министерство финансов заявило, что не будет погашать и выплачивать купонный доход по облигациям на общую сумму $24,22 миллиона, которые до этого были украд


о словам, намеченный на 5 августа конкурс состоится, как и предполагалось, и не будет перенесен или отменен. Прием заявок на участие в конкурсе по акциям Норильского Никеля проводился с 22 июля по 16.


 хотят немного подождать, посмотреть и купить уже, может быть, что-то подешевле после понижения, но гнать его (рынок) до бесконечности никто не будет", - сказал он. "Посмотрим, как покажет

In [33]:
with open("tags_without_no.txt", "w") as f:
    for context in hasnt_negative_word_contexts:
        f.write(context.replace("\x1b[31m", "**").replace("\x1b[0m", "**"))
        f.write("\n\n")

In [27]:
%%time

hasnt_negative_tag_count = 0  # количество вхождений "не" и "нет", вокруг которых нет тега
hasnt_negative_tag_contexts = []  # контексты этих слов
not_count = 0  # всего вхождений частицы не
files_count = 0  # счетчик файлов, нужен для вывода прогресса

for filename in directory:
    
    data = pd.read_csv(filename, sep='\t', names=['Offset', 'Text', 'Tags'], skip_blank_lines=False)
        
    for row_number in range(len(data)):
        
        text = data['Text'][row_number]
        # Находим "не" и "нет"
        if not isinstance(text, float):
            text = text.split("=")[1].lower()
            if text == "не" or text == "нет":
            
                not_count += 1
                has_negative_tag = False

                # Смотрим на его +/-3 окно
                for window_step in [-3, -2, -1, 0, 1, 2, 3]:
                    if row_number + window_step >= 0 and row_number + window_step < len(data) and \
                        not pd.isnull(data['Tags'][row_number + window_step]):

                        has_negative_tag = True

                if (not has_negative_tag and not isinstance(text, float)):
                    hasnt_negative_tag_count += 1
                    offset = int(data['Offset'][row_number].split("=")[1])
                    file_text = cat(filename.replace("csv", "txt"))
                    hasnt_negative_tag_contexts.append(file_text[offset - int(window_size / 2):offset] + 
                                                       '\x1b[31m' + file_text[offset:offset + len(text)] + 
                                                       '\x1b[0m' + file_text[offset + len(text):offset + 
                                                                             int(window_size / 2)])
            
    files_count += 1
    if files_count % 1000 == 0:
        print(files_count)

1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
CPU times: user 1min 16s, sys: 964 ms, total: 1min 17s
Wall time: 1min 24s


### Вхождения "не" и "нет", вокруг которых в окне +/- 3 нет тега отрицания

In [28]:
hasnt_negative_tag_count

544

In [29]:
# Всего вхождений "не" и "нет"
not_count

21792

In [30]:
hasnt_negative_tag_count / not_count

0.024963289280469897

In [31]:
for context in hasnt_negative_tag_contexts[:10]:
    print(context.replace("\x1b[31m", "\x1b[35m"))
    print("\n")

т Грузии Эдуард Шеварднадзе принял президента NPL Брюса Раппапорта. На встрече Шеварднадзе высказал неудовлетворение темпами добычи нефти и подчеркнул необходимость пересмотра условий соглашения между


не одноместный. МОСКВА, 18 авг (Рейтер) - В подмосковном городе Жуковском разбился транспортный шест


аты за газ, в Белоруссии внедряется новая система, предусматривающая конвертацию белорусских рублей не через Межбанковскую Валютную Бирже (МВБ), а через российские субъекты хозяйствования. Будут расши


 год 5-6 процентов, сказал вице-премьер Виктор Пинзеник. "В нынешних условиях, пока Верховный Совет не принял пакет законопроектов по реформированию налоговой системы, мы можем прогнозировать сокращен


но обсудить этот вопрос с премьером Лазаренко. При этом президент, с одной стороны, подчеркнул свое неудовлетворение работой экономического блока правительства, а с другой - сослался на слова Пинзеник


ионеры, получающие дополнительные акции в результате переоценки основных фондов предп

In [35]:
with open("no_without_tags.txt", "w") as f:
    for context in hasnt_negative_tag_contexts:
        f.write(context.replace("\x1b[31m", "**").replace("\x1b[0m", "**"))
        f.write("\n\n")